In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import dictys
from dictys.net import stat
import joblib
import pickle
from scipy.stats import median_abs_deviation, hypergeom
import math

In [2]:
from utils_custom import *
from episodic_dynamics import *
from pseudotime_curves import *

In [3]:
# Paths
dictys_dynamic_object_path = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added_v2/output/dynamic.h5"
output_folder = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added_v2/output/intermediate_tmp_files/blimp1_ko/gc_98'
latent_factor_folder = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/other_files/latent_factors'
lf_blimp1_file = f"{latent_factor_folder}/feature_list_Z5_PRDM1_KO.txt"
lf_irf4_file = f"{latent_factor_folder}/feature_list_Z4_IRF4_KO.txt"

In [4]:
lf_blimp1 = pd.read_csv(lf_blimp1_file, sep='\t')['names'].tolist()
lf_irf4 = pd.read_csv(lf_irf4_file, sep='\t')['names'].tolist()

In [6]:
run_episode(
    episode_idx=4,
    dictys_dynamic_object_path=dictys_dynamic_object_path,
    output_folder=output_folder,
    latent_factor_folder=latent_factor_folder,
    trajectory_range=(1, 3),
    num_points=40,
    time_slice_start=15,
    time_slice_end=20, # last index is excluded
    lf_genes=lf_blimp1
)

KeyboardInterrupt: 

# Un-parallelized version

In [ ]:
# Load data
dictys_dynamic_object = dictys.net.dynamic_network.from_file(dictys_dynamic_object_path)

### Expression dynamics

In [ ]:
# get lcpm chars for these genes
lcpm_dcurve, dtime = compute_expression_regulation_curves(dictys_dynamic_object, start=1, stop=3, num=40, dist=0.001, mode="expression")
# remove genes with names starting with ZNF and ZBTB
lcpm_dcurve = lcpm_dcurve[~lcpm_dcurve.index.str.startswith('ZNF') & ~lcpm_dcurve.index.str.startswith('ZBTB')]
# get lcpm chars for these genes
#lcpm_dcurve_gc, dtime_gc = compute_expression_regulation_curves(dictys_dynamic_object, start=0, stop=3, num=20, dist=0.001, mode="expression")
# slice the dcurve for the lf genes using gene names which are indices in pandas df
display(lcpm_dcurve.head())

### Regulation dynamics


In [ ]:
pts, fsmooth = dictys_dynamic_object.linspace(1,3,40,0.001)
stat1_net = fsmooth(stat.net(dictys_dynamic_object)) #varname=w_in loads total effect network
stat1_netbin = stat.fbinarize(stat1_net,sparsity=0.01)
stat1_x=stat.pseudotime(dictys_dynamic_object,pts)
dtime = pd.Series(stat1_x.compute(pts)[0])

# Get episode specific GRN (transient state specific)

In [ ]:
# pts is a dictys traj object
dnetbin = stat1_netbin.compute(pts)
dnetbin_episode = dnetbin[:, :, 0:5] #5 timepoints, excludes the last one

In [ ]:
# compute the weighted network
dnet = stat1_net.compute(pts)
dnet_episode = dnet[:, :, 0:5]
display(dnet_episode)

### Filter episodic GRN edges which are significantly non-zero across time points and are direction invariant

In [ ]:
###### Get the tf and target names #####
# Create reverse mapping: index -> gene_name
ndict = dictys_dynamic_object.ndict
index_to_gene = {idx: name for name, idx in ndict.items()}
target_names = [index_to_gene[idx] for idx in range(dnetbin_episode.shape[1])]
# Get TF_gene_indices from TFs_to_keep_indices using nids[0]
tf_gene_indices = [dictys_dynamic_object.nids[0][tf_idx] for tf_idx in range(dnetbin_episode.shape[0])]
tf_names = [index_to_gene[idx] for idx in tf_gene_indices]
print(len(target_names))
print(len(tf_names))

In [ ]:
###### Create multi-index tuples (all combinations of TF-target pairs) ######
index_tuples = [(tf, target) for tf in tf_names for target in target_names]
multi_index = pd.MultiIndex.from_tuples(index_tuples, names=['TF', 'Target'])
# Reshape the subnetworks array to 2D (pairs × time points)
n_tfs, n_targets, n_times = dnet_episode.shape
reshaped_data = dnet_episode.reshape(-1, n_times)
# Create DataFrame with multi-index
episode_beta_dcurve = pd.DataFrame(
    reshaped_data,
    index=multi_index,
    columns=[f'time_{i}' for i in range(n_times)]
)
# drop rows that are all 0
episode_beta_dcurve = episode_beta_dcurve[episode_beta_dcurve.sum(axis=1) != 0]
# the number of edges here remain the same between episodes, indicating that exact 0 are the edges that don't have atac-seq basis
display(episode_beta_dcurve.head())
display(episode_beta_dcurve.shape)

In [ ]:
###### Filtering the global episodic GRN for retaining significantly non-zero and direction invariant edges ######
filtered_edges = filter_edges_by_significance_and_direction(
    episode_beta_dcurve,
    min_nonzero_timepoints=3,
    alpha=0.05,
    min_observations=3,
    check_direction_invariance=True,  # Enable direction invariance check
    n_processes=60,
    chunk_size=8000,
    save_intermediate=False,
    intermediate_path=output_folder
)
    
print(f"\nFinal shape: {filtered_edges.shape}")

In [ ]:
# load episode parquet file into pandas
#filtered_edges = pd.read_parquet('/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added_v2/output/intermediate_tmp_files/filtered_edges_significant_invariant_PB_ep4.parquet')
# remove TFs starting with ZNF or ZBTB
filtered_edges = filtered_edges[~filtered_edges.index.get_level_values(0).str.startswith('ZNF') & ~filtered_edges.index.get_level_values(0).str.startswith('ZBTB')]
display(filtered_edges)

In [ ]:
# drop edges with p_value > 0.01
filtered_edges_p001 = filtered_edges[filtered_edges['p_value'] < 0.001]
display(filtered_edges_p001.head())
display(filtered_edges_p001.shape)

## Get TF forces for the episode & State

In [ ]:
###### CHANGE STATE HERE ######
tf_lcpm_values = lcpm_dcurve.loc[filtered_edges_p001.index.get_level_values(0).unique()]
display(tf_lcpm_values.shape)
###### CHANGE EPISODE HERE ######
tf_lcpm_episode = tf_lcpm_values.iloc[:, 15:20]

# Adaptive column renaming to match beta curves
beta_time_cols = [col for col in filtered_edges_p001.columns if col.startswith('time_')]
n_time_cols = len(beta_time_cols)

# Rename TF expression columns to match beta curves format
tf_lcpm_episode.columns = beta_time_cols[:n_time_cols]
display(tf_lcpm_episode)

In [ ]:
# Prepare beta curves (only time columns)
beta_curves_for_force = filtered_edges_p001.drop('p_value', axis=1)

print("Starting parallel force calculation...")
print(f"Input beta curves shape: {beta_curves_for_force.shape}")
print(f"TF expression shape: {tf_lcpm_episode.shape}")

# Calculate forces in parallel
force_curves = calculate_force_curves_parallel(
    beta_curves=beta_curves_for_force,
    tf_expression=tf_lcpm_episode,
    n_processes=20,  # Adjust based on your system
    chunk_size=30000,  # Adjust based on available memory
    epsilon=1e-10,
    save_intermediate=False
)

print(f"Force calculation completed!")
print(f"Output shape: {force_curves.shape}")
# Display sample results
display(force_curves.head())

#### Get the average force over the episode

In [ ]:
# Calculate average force over the 5 time points
avg_force = force_curves.mean(axis=1)
print(f"Average force shape: {avg_force.shape}")
# Convert to DataFrame with proper column name
avg_force_df = avg_force.to_frame(name='avg_force')
display(avg_force_df.head())

In [ ]:
# Plot distribution
fig, axes = plt.subplots(1, 1, figsize=(15, 6))

# Separate positive and negative forces
positive_forces = avg_force[avg_force > 0]
negative_forces = avg_force[avg_force < 0]

axes.hist([positive_forces, negative_forces], bins=50, alpha=0.7, 
              color=['red', 'blue'], label=['Positive Forces', 'Negative Forces'], edgecolor='black')
axes.set_xlabel('Average Force')
axes.set_ylabel('Number of Edges')
axes.set_title('Distribution by Force Direction')
axes.legend()
axes.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd

if len(positive_forces) > 0:
    pos_threshold = np.percentile(positive_forces, 98)
    top_pos_edges = positive_forces[positive_forces >= pos_threshold]
else:
    top_pos_edges = pd.Series(dtype=avg_force.dtype)

if len(negative_forces) > 0:
    neg_threshold = np.percentile(negative_forces, 0.5)
    top_neg_edges = negative_forces[negative_forces <= neg_threshold]
else:
    top_neg_edges = pd.Series(dtype=avg_force.dtype)

# Combine into a DataFrame
episodic_grn_edges = pd.concat([top_pos_edges, top_neg_edges]).to_frame(name='avg_force')

# Optionally, sort by force value (descending for positive, ascending for negative)
episodic_grn_edges = episodic_grn_edges.sort_values(by='avg_force', ascending=False)

# Display or use as needed
print(episodic_grn_edges)
print(f"Number of positive edges: {len(top_pos_edges)}")
print(f"Number of negative edges: {len(top_neg_edges)}")
print(f"Total selected edges: {len(episodic_grn_edges)}")
# display the unique tf and target numbers
display(len(episodic_grn_edges.index.get_level_values(0).unique()))
display(len(episodic_grn_edges.index.get_level_values(1).unique()))

#### Select the top k% of edges to build the episodic GRN

## Get the TFs acting on LF genes

In [ ]:
# load LF files 
z11_file = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/other_files/latent_factors/feature_list_Z11_GC_PB.txt'
z3_file = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/other_files/latent_factors/feature_list_Z3_GC_PB.txt'
z_prdm1_ko = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/other_files/latent_factors/feature_list_Z5_PRDM1_KO.txt"
# load into a list of gene names 
z11 = pd.read_csv(z11_file, sep='\t', header=0)
z3 = pd.read_csv(z3_file, sep='\t', header=0)
z_prdm1_ko = pd.read_csv(z_prdm1_ko, sep='\t', header=0)
# remove HLA- genes
z11 = z11[~z11['names'].str.contains('HLA-')]
z3 = z3[~z3['names'].str.contains('HLA-')]
z_prdm1_ko = z_prdm1_ko[~z_prdm1_ko['names'].str.contains('HLA-')]

In [ ]:
# get the gene names 
z11_genes = z11['names'].tolist()
z3_genes = z3['names'].tolist()
z_prdm1_ko_genes = z_prdm1_ko['names'].tolist()
# create a list of all lf genes 
#lf_genes = list(set(z11_genes + z3_genes))
lf_genes = list(set(z_prdm1_ko_genes))
lf_in_object = check_if_gene_in_ndict(dictys_dynamic_object, lf_genes, return_index=True)
print(f"Found {len(lf_in_object['present'])} genes")
print(f"Missing {len(lf_in_object['missing'])} genes")
print("Indices:", lf_in_object['indices'])

#### Get the enrichment (over representation) of LF genes in TF regulons

In [ ]:
# add a boolean column indicating if the target is a lf gene in the episodic grn df
episodic_grn_edges['is_in_lf'] = episodic_grn_edges.index.get_level_values(1).isin(lf_genes)
# get the number of genes in the episodic grn edges
target_genes_in_episodic_grn = episodic_grn_edges.index.get_level_values(1).unique()
display("Number of targets in episodic grn: ", len(target_genes_in_episodic_grn))
# get the number of lf_genes in the episodic grn edges
lf_in_episodic_grn = episodic_grn_edges[episodic_grn_edges['is_in_lf']]
display(lf_in_episodic_grn.head())
# get the unique LF genes active in the lf_in_episodic_grn df
lf_genes_active_in_episode = lf_in_episodic_grn.index.get_level_values(1).unique()
display("Number of LF genes active in episode: ", len(lf_genes_active_in_episode))
tfs_acting_on_lf = lf_in_episodic_grn.index.get_level_values(0).unique()
display("TFs acting on LF genes: ", tfs_acting_on_lf, len(tfs_acting_on_lf))
# take the tfs in the lf_in_episodic_grn df and subset the episodic_grn_edges df to only include these tfs
episodic_grn_edges_subset = episodic_grn_edges[episodic_grn_edges.index.get_level_values(0).isin(lf_in_episodic_grn.index.get_level_values(0))]
display(episodic_grn_edges_subset.head())
display(episodic_grn_edges_subset.shape)

In [ ]:
episodic_enrichment_df = calculate_tf_episodic_enrichment(episodic_grn_edges_subset, 
                                       len(lf_genes_active_in_episode), 
                                       len(target_genes_in_episodic_grn))
# sort the episodic_enrichment_df_p005 by enrichment_score in descending order
episodic_enrichment_df_sorted = episodic_enrichment_df.sort_values(by='enrichment_score', ascending=False)
# filter the episodic_enrichment_df to only include tfs with a p_value < threshold
episodic_enrichment_df_p005 = episodic_enrichment_df_sorted[episodic_enrichment_df_sorted['p_value'] < 0.05]
display(episodic_enrichment_df_p005)

In [ ]:
# save the episodic_enrichment_df_p005_sorted df to a csv file
episodic_enrichment_df_sorted.to_csv(os.path.join(output_folder, 'enrichment_ep1_blimp_ko.csv'), index=False)